<a href="https://colab.research.google.com/github/aithaprasad/NLP-News-Classification/blob/main/NLP_Telugu_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers import Embedding, Input
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [ ]:
telugu_df = pd.read_csv("train_telugu_news.csv")

In [ ]:
test_df = pd.read_csv("test_telugu_news.csv")

In [ ]:
telugu_df.head()

,SNo,date,heading,body,topic
0,414,11-05-2017 00:39:13,ఐడిబిఐపై ఆర్‌బిఐ నజర్‌,భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ ...,business
1,2668,12-06-2017 00:40:32,బ్యాంకింగ్‌ చీఫ్‌లతో నేడు జైట్లీ భేటీ,న్యూఢిల్లీ : ఆర్థిక మంత్రి అరుణ్‌ జైట్లీ సోమవా...,business
2,19923,19-01-2017 19:51:31,కీలక వికెట్ తీసిన జడేజా..,కటక్: ఇంగ్లండ్‌తో జరుగుతున్న సెకండ్ వన్డే మ్యా...,sports
3,15762,23-11-2017 17:29:04,మరో రెచ్చగొట్టే చర్యకు దిగిన పాకిస్థాన్,\nఇస్లామాబాద్ : పాకిస్థాన్ అంతర్జాతీయ ఉగ్రవాది...,nation
4,8624,03-04-2017 15:48:23,గోవాలో కొడుకుతో కలిసి అల్లు అర్జున్ స్విమ్మింగ్!,స్టార్‌ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటి...,entertainment


In [ ]:
all_topics = set(telugu_df['topic'])

In [ ]:
all_topics

{'business', 'editorial', 'entertainment', 'nation', 'sports'}

In [ ]:
telugu_df.shape

(17312, 5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(telugu_df['body'], telugu_df['topic'], test_size=0.1, random_state=42)

In [ ]:
X_train.shape

(15580,)

In [ ]:
X_val.shape

(1732,)

In [ ]:
X_test, y_test = test_df['body'], test_df['topic']

In [ ]:
X_test.shape

(4329,)

In [ ]:
mini, maxi = 10000, 0
mini_count, maxi_count = 0, 0
for sentence in X_train:
  mini = min(len(sentence), mini)
  maxi = max(len(sentence), mini)
  if len(sentence) < 500: mini_count += 1
  else: maxi_count += 1
print(mini)
print(maxi)
print(mini_count)
print(maxi_count)

6
2087
2481
13099


In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

X_train_tok = tokenizer.texts_to_sequences(X_train)
X_val_tok = tokenizer.texts_to_sequences(X_val)

In [ ]:
X_test_tok = tokenizer.texts_to_sequences(X_test)

In [ ]:
len(tokenizer.word_index)

295592

In [ ]:
from keras_preprocessing.sequence import pad_sequences
vocab_size = len(tokenizer.word_index) + 1

maxlen = 500

X_train_pad = pad_sequences(X_train_tok, padding='post', maxlen=maxlen)
X_val_pad = pad_sequences(X_val_tok, padding='post', maxlen=maxlen)

In [ ]:
X_test_pad = pad_sequences(X_test_tok, padding='post', maxlen=maxlen)

In [ ]:
def create_model(output_dim, dropout=None):
  model = Sequential()
  embedding_layer = Embedding(vocab_size, output_dim, input_length=maxlen)
  model.add(embedding_layer)
  if dropout: model.add(Dropout(dropout))
  model.add(Flatten())
  model.add(Dense(5, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
  print(model.summary())
  return model

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 500)          147796500 
                                                                 
 dropout_1 (Dropout)         (None, 500, 500)          0         
                                                                 
 flatten_1 (Flatten)         (None, 250000)            0         
                                                                 
 dense_1 (Dense)             (None, 5)                 1250005   
                                                                 
Total params: 149,046,505
Trainable params: 149,046,505
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from keras.utils import to_categorical
from collections import defaultdict

data = list(all_topics)
dic = defaultdict(int)
counter = 0
for val in data:
    if not dic[val]: #if its a new entry add it to the dict and increase the counter
        print(val, counter)
        dic[val] = counter
        counter+=1

nation 0
entertainment 1
sports 2
editorial 3
business 4


In [ ]:
labels = [dic[x] for x in y_train]

In [ ]:
dev_labels = [dic[x] for x in y_val]

In [ ]:
import tensorflow as tf
y_categorical = tf.keras.utils.to_categorical(labels, num_classes=5)

In [ ]:
y_val_categorical = tf.keras.utils.to_categorical(dev_labels, num_classes=5)

I fine-tuned hyper parameters and finally figured out the best validation accuracy is with batch size as 32, epochs 2.

I have tried on changing the output_dim of embedding layer, tried without dropout and then increased dropout. Best is with Dropout of 2. 

Tried batch_sizes of [5, 16, 32, 64, 128] with epochs ranging between [2, 20] with a skip of 2 each time.

In [ ]:
def hyper_param_tune(batch_size, epoch):
  model = create_model(500, 0.2)
  history = model.fit(X_train_pad, y_categorical, batch_size=32, epochs=2, verbose=1, validation_data=(X_val_pad, y_val_categorical))
  print(history.history['val_acc'])

In [ ]:
best_params = {'batch_size': 32, 'epochs' : 2, 'dropout' : 0.2, 'output_dim' : 500}

In [ ]:
model = create_model(best_params['output_dim'], best_params['dropout'])
history = model.fit(X_train_pad, y_categorical, batch_size=best_params['batch_size'], epochs=best_params['epochs'], verbose=1, validation_data=(X_val_pad, y_val_categorical))

Epoch 1/2
487/487 [==============================] - 41s 84ms/step - loss: 0.4146 - acc: 0.8689 - val_loss: 0.1449 - val_acc: 0.9544
Epoch 2/2
487/487 [==============================] - 41s 84ms/step - loss: 0.0215 - acc: 0.9944 - val_loss: 0.1542 - val_acc: 0.9515


In [ ]:
model.save('drop_two_ep_2_bs_32.h5')

In [ ]:
from sklearn import metrics
import numpy as np

y_final_test_predicted_labels = model.predict(X_test_pad)
y_final_test_predicted_labels = np.argmax(y_final_test_predicted_labels, axis=1) 

final_test_accuracy = metrics.accuracy_score(y_test, y_final_test_predicted_labels)

136/136 [==============================] - 0s 2ms/step


In [ ]:
y_test_labels = []
for i in y_test:
  y_test_labels.append(dic[i])

In [ ]:
final_test_accuracy = metrics.accuracy_score(y_test_labels, y_final_test_predicted_labels)

In [ ]:
final_test_accuracy

0.957033957033957

In [ ]:
final_test_f1score = metrics.f1_score(y_test_labels, y_final_test_predicted_labels, average='weighted')

In [ ]:
final_test_f1score

0.9561872563665075